In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats

/opt/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
social_factors = pd.read_csv("SeattleData/SeattleSocialFactors_1.csv")
social_factors.head()

Geo_FIPS  family_ppl%  ttw_30_59%  ttw_60_89%   ttw_90%  ttw_bycar%  \
0  530330001001     0.805365    0.248281    0.000000  0.000000    0.365887   
1  530330001002     0.666436    0.183011    0.015884  0.060083    0.306630   
2  530330001003     0.593532    0.314521    0.022194  0.000000    0.424857   
3  530330001004     0.608620    0.124636    0.056494  0.000000    0.367501   
4  530330001005     0.396004    0.253406    0.023615  0.000000    0.345141   

   ttw_bytransit%  ttw_bybike%  ttw_byfoot%  median_vehicles_per_household  \
0        0.070839     0.026823     0.000000                            1.6   
1        0.185773     0.000000     0.000000                            1.6   
2        0.103995     0.019023     0.000000                            1.6   
3        0.147932     0.000000     0.012813                            1.7   
4        0.184378     0.015441     0.020890                            1.6   

   ppl_growth%_2010_2020  ppl_growth%_2020_2025  median_rent  \
0                  16.32                   4.40         1647   
1                  17.34                   4.28          821   
2                  17.95                   4.57          797   
3                  15.08                   4.37          674   
4                  16.88                   4.54          989   

   income_household_median  income_percapita  quality_of_life_score  \
0                   154138             71251                    133   
1                    30978             23190                    123   
2                    66118             52181                    119   
3                    34758             22211                    124   
4                    57292             41977                    119   

   daytime_ppl  transport_expenditure%  vehicle_purchase%      gas%  \
0         1035                0.149076           0.404101  0.173287   
1         1004                0.157356           0.391732  0.204916   
2         1321                0.152064           0.393374  0.189434   
3         1700                0.158326           0.387533  0.208966   
4         1007                0.152415           0.386423  0.193007   

   vehicle_others%  public_transport%  ppl_density  ppl_density_change  \
0         0.310203           0.118847       4209.9               590.7   
1         0.326656           0.077755      13004.2              1921.9   
2         0.320162           0.100150      11017.0              1676.7   
3         0.332019           0.072405      35176.5              4609.6   
4         0.320597           0.095382      12370.6              1786.4   

   ppl_density_change%  hs_1_detached%  hs_1_attached%     hs_2%   hs_3_4%  \
0             0.140312        1.000000        0.000000  0.000000  0.000000   
1             0.147791        0.147059        0.043417  0.060224  0.060224   
2             0.152192        0.302583        0.066421  0.000000  0.146371   
3             0.131042        0.060706        0.039735  0.000000  0.025386   
4             0.144407        0.127596        0.000000  0.000000  0.000000   

    hs_5_9%  hs_10_19%  hs_20_49%   hs_50p%  mortality_index  murder_index  \
0  0.000000   0.000000   0.000000  0.000000              104            26   
1  0.135854   0.054622   0.100840  0.397759               93           196   
2  0.067651   0.094711   0.017220  0.305043               95           194   
3  0.161148   0.162252   0.320088  0.195364               22           197   
4  0.037092   0.000000   0.106825  0.728487              150           187   

   rape_index  robbery_index  assault_index  burglary_index  larceny_index  \
0          13              8             74              71            123   
1         197            185            196             147            188   
2         193            174            192             124            180   
3         196            186            194             147            189   
4         168            194            182             1

In [4]:
social_factors = social_factors.rename(columns={'Geo_FIPS': 'GEOID10'})

In [5]:
bg = gpd.read_file('SeattleData/Census_Block_Groups_2010/Census_Block_Groups_2010.shp')
bg.head()

OBJECTID TRACTCE10 BLKGRPCE10       GEOID10     NAMELSAD10   INTPTLAT10  \
0         1    000100          1  530330001001  Block Group 1  +47.7276870   
1         2    000100          2  530330001002  Block Group 2  +47.7293165   
2         3    000100          3  530330001003  Block Group 3  +47.7228093   
3         4    000100          4  530330001004  Block Group 4  +47.7319367   
4         5    000100          5  530330001005  Block Group 5  +47.7229283   

     INTPTLON10  TRACT   TRBG TRBG_STR  ACRES_TOTA  ACRES_LAND  ACRES_WATE  \
0  -122.2815160    100  100.1    100.1  588.283002  221.041200  367.285577   
1  -122.2924690    100  100.2    100.2   71.258030   71.263215    0.000000   
2  -122.2894558    100  100.3    100.3   91.604629   91.611355    0.000000   
3  -122.2944531    100  100.4    100.4   31.237245   31.239309    0.000000   
4  -122.2944374    100  100.5    100.5   56.956546   56.960756    0.000000   

   WATER    SHAPE_Leng    SHAPE_Area  \
0    NaN  21168.918543  2.562561e+07   
1    NaN   8692.089186  3.104000e+06   
2    NaN   8560.645769  3.990298e+06   
3    NaN   4693.439609  1.360694e+06   
4    NaN   7450.759251  2.481027e+06   

                                            geometry  
0  POLYGON ((-122.26840 47.72641, -122.26719 47.7...  
1  POLYGON ((-122.28970 47.73193, -122.28965 47.7...  
2  POLYGON ((-122.28633 47.72093, -122.28629 47.7...  
3  POLYGON ((-122.29237 47.73194, -122.29231 47.7...  
4  POLYGON ((-122.29273 47.72019, -122.29273 47.7...

In [6]:
bg_geo = bg[['GEOID10','geometry']]

In [7]:
bg_geo=bg_geo.astype({'GEOID10': 'int64'})

In [8]:
social_geo = social_factors.merge(bg_geo, how='inner', on='GEOID10')

In [10]:
social_geo = gpd.GeoDataFrame(social_geo, geometry='geometry')

In [22]:
social_geo.explore(tiles="CartoDB positron",
                  column="ttw_bycar%",
                  scheme="naturalbreaks",
                  style_kwds=dict(stroke=False),
                tooltip="ttw_bycar%" )

In [25]:
social_geo.explore(tiles="CartoDB positron",
                  column="quality_of_life_score",
                  scheme="naturalbreaks",
                  style_kwds=dict(stroke=False),
                tooltip="quality_of_life_score" ,
                  cmap='GnBu_r')